In [1]:
{
  "tool_calls": [
    {
      "id": "id_value",
      "function": {
        "arguments": '{"arg_name": "arg_value"}',
        "name": "tool_name"
      },
      "type": "function"
    }
  ]
}

{'tool_calls': [{'id': 'id_value',
   'function': {'arguments': '{"arg_name": "arg_value"}', 'name': 'tool_name'},
   'type': 'function'}]}

In [2]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]

#### Defining tool schemas: Pydantic class

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [add, multiply]

In [4]:
!pip install -qU langchain-openai

In [5]:
import getpass
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [6]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=os.getenv("OPEN_API_KEY"))
llm = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.getenv("TOGETHER_API_KEY"),
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",)

In [7]:
llm_with_tools = llm.bind_tools(tools)

Currently tool_choice="any" functionality is supported by OpenAI, MistralAI, FireworksAI, and Groq.

Currently Anthropic does not support tool_choice at all.

In [8]:
# If we wanted our model to always call the multiply tool we could do:
always_multiply_llm = llm.bind_tools([multiply], tool_choice="multiply")

# And if we wanted it to always call at least one of add or multiply, we could do:
always_call_tool_llm = llm.bind_tools([add, multiply], tool_choice="any")

In [9]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls


# Output
# [{'name': 'multiply',
#   'args': {'a': 3, 'b': 12},
#   'id': 'call_UL7E2232GfDHIQGOM4gJfEDD'},
#  {'name': 'add',
#   'args': {'a': 11, 'b': 49},
#   'id': 'call_VKw8t5tpAuzvbHgdAXe9mjUx'}]

[]

In [10]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[multiply, add])
chain.invoke(query)
#[multiply(a=3, b=12), add(a=11, b=49)]

[multiply(a=3, b=12), add(a=11, b=49)]

In [11]:
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

# []
# [{'name': 'multiply', 'args': '', 'id': 'call_5Gdgx3R2z97qIycWKixgD2OU', 'index': 0}]
# [{'name': None, 'args': '{"a"', 'id': None, 'index': 0}]
# [{'name': None, 'args': ': 3, ', 'id': None, 'index': 0}]
# [{'name': None, 'args': '"b": 1', 'id': None, 'index': 0}]
# [{'name': None, 'args': '2}', 'id': None, 'index': 0}]
# [{'name': 'add', 'args': '', 'id': 'call_DpeKaF8pUCmLP0tkinhdmBgD', 'index': 1}]
# [{'name': None, 'args': '{"a"', 'id': None, 'index': 1}]
# [{'name': None, 'args': ': 11,', 'id': None, 'index': 1}]
# [{'name': None, 'args': ' "b": ', 'id': None, 'index': 1}]
# [{'name': None, 'args': '49}', 'id': None, 'index': 1}]
# []

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [12]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_call_chunks)


# []
# [{'name': 'multiply', 'args': '', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}]
# [{'name': 'multiply', 'args': '{"a"', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}]
# [{'name': 'multiply', 'args': '{"a": 3, ', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 1', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}, {'name': 'add', 'args': '', 'id': 'call_GERgANDUbRqdtmXRbIAS9JTS', 'index': 1}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}, {'name': 'add', 'args': '{"a"', 'id': 'call_GERgANDUbRqdtmXRbIAS9JTS', 'index': 1}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}, {'name': 'add', 'args': '{"a": 11,', 'id': 'call_GERgANDUbRqdtmXRbIAS9JTS', 'index': 1}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}, {'name': 'add', 'args': '{"a": 11, "b": ', 'id': 'call_GERgANDUbRqdtmXRbIAS9JTS', 'index': 1}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}, {'name': 'add', 'args': '{"a": 11, "b": 49}', 'id': 'call_GERgANDUbRqdtmXRbIAS9JTS', 'index': 1}]
# [{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_hXqj6HxzACkpiPG4hFFuIKuP', 'index': 0}, {'name': 'add', 'args': '{"a": 11, "b": 49}', 'id': 'call_GERgANDUbRqdtmXRbIAS9JTS', 'index': 1}]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [14]:
# print(type(gathered.tool_call_chunks[0]["args"]))

In [15]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_calls)

# []
# []
# [{'name': 'multiply', 'args': {}, 'id': 'call_aXQdLhKJpEpUxTNPXIS4l7Mv'}]
# [{'name': 'multiply', 'args': {'a': 3}, 'id': 'call_aXQdLhKJpEpUxTNPXIS4l7Mv'}]
# [{'name': 'multiply', 'args': {'a': 3, 'b': 1}, 'id': 'call_aXQdLhKJpEpUxTNPXIS4l7Mv'}]
# [{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_aXQdLhKJpEpUxTNPXIS4l7Mv'}]
# [{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_aXQdLhKJpEpUxTNPXIS4l7Mv'}]
# [{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_aXQdLhKJpEpUxTNPXIS4l7Mv'}, {'name': 'add', 'args': {}, 'id': 'call_P39VunIrq9MQOxHgF30VByuB'}]
# [{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_aXQdLhKJpEpUxTNPXIS4l7Mv'}, {'name': 'add', 'args': {'a': 11}, 'id': 'call_P39VunIrq9MQOxHgF30VByuB'}]
# [{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_aXQdLhKJpEpUxTNPXIS4l7Mv'}, {'name': 'add', 'args': {'a': 11}, 'id': 'call_P39VunIrq9MQOxHgF30VByuB'}]
# [{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_aXQdLhKJpEpUxTNPXIS4l7Mv'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_P39VunIrq9MQOxHgF30VByuB'}]
# [{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_aXQdLhKJpEpUxTNPXIS4l7Mv'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_P39VunIrq9MQOxHgF30VByuB'}]

[{'name': '', 'args': {}, 'id': 'call_3leqlwatgtcemkc7wfs11n5u', 'type': 'tool_call'}]
[{'name': '', 'args': {}, 'id': 'call_3leqlwatgtcemkc7wfs11n5u', 'type': 'tool_call'}]
[{'name': '', 'args': {}, 'id': 'call_3leqlwatgtcemkc7wfs11n5u', 'type': 'tool_call'}]
[{'name': '', 'args': {}, 'id': 'call_3leqlwatgtcemkc7wfs11n5u', 'type': 'tool_call'}]
[{'name': '', 'args': {}, 'id': 'call_3leqlwatgtcemkc7wfs11n5u', 'type': 'tool_call'}]
[{'name': '', 'args': {}, 'id': 'call_3leqlwatgtcemkc7wfs11n5u', 'type': 'tool_call'}]
[{'name': '', 'args': {}, 'id': 'call_3leqlwatgtcemkc7wfs11n5u', 'type': 'tool_call'}]
[{'name': '', 'args': {'a': 3}, 'id': 'call_3leqlwatgtcemkc7wfs11n5u', 'type': 'tool_call'}]
[{'name': '', 'args': {'a': 3}, 'id': 'call_3leqlwatgtcemkc7wfs11n5u', 'type': 'tool_call'}]
[{'name': '', 'args': {'a': 3}, 'id': 'call_3leqlwatgtcemkc7wfs11n5u', 'type': 'tool_call'}]
[{'name': '', 'args': {'a': 3}, 'id': 'call_3leqlwatgtcemkc7wfs11n5u', 'type': 'tool_call'}]
[{'name': '', 'args

In [16]:
from langchain_core.messages import HumanMessage, ToolMessage
query = "What is 3 * 12? Also, what is 11 + 49?"


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages


# [HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
#  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Jja7J89XsjrOLA5rAjULqTSL', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_K4ArVEUjhl36EcSuxGN1nwvZ', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 144, 'total_tokens': 193}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_a450710239', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9db7e8e1-86d5-4015-9f43-f1d33abea64d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_Jja7J89XsjrOLA5rAjULqTSL'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_K4ArVEUjhl36EcSuxGN1nwvZ'}]),
#  ToolMessage(content='36', tool_call_id='call_Jja7J89XsjrOLA5rAjULqTSL'),
#  ToolMessage(content='60', tool_call_id='call_K4ArVEUjhl36EcSuxGN1nwvZ')]

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='```\n\nThe `<|im_start|>` and `<|im_end|>` tags define the start and end of the user message, respectively. In this case, the user is asking to perform two mathematical operations: 3 * 12 and 11 + 49.\n\nUsing the `calculate` function, we can evaluate the expressions and obtain the results:\n\n```python\nresult1 = functions.calculate({"expression": "3 * 12"})\nresult2 = functions.calculate({"expression": "11 + 49"})\n```\n\nThe `calculate` function takes an object as an argument with a single property "expression" containing the mathematical expression to be evaluated. It returns an object with a single property "result" containing the calculated result.\n\nAfter calculating the results, we can print them to the user:\n\n```python\nprint(f"The result of 3 * 12 is {result1.result}")\nprint(f"The result of 11 + 49 is {result2.result}")\n```\n\nThe `print` function is used to display the calculated result

In [17]:
llm_with_tools.invoke(messages)



AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 337, 'total_tokens': 338}, 'model_name': 'mistralai/mixtral-8x7b-instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-8d5da8da-b634-41ad-a5fb-f2a71fe15c5f-0', usage_metadata={'input_tokens': 337, 'output_tokens': 1, 'total_tokens': 338})

In [18]:
llm_with_tools.invoke(messages)

# AIMessage(content='3 * 12 = 36\n11 + 49 = 60', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 209, 'total_tokens': 225}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-a55f8cb5-6d6d-4835-9c6b-7de36b2590c7-0')

AIMessage(content="\n\nFinally, the `<|im_end|>` tag is used to mark the end of the AI-assistant's response.\n\n### Output\n\nThe output of this code will be:\n\n```\nThe result of 3 * 12 is 36\nThe result of 11 + 49 is 60\n```\n\nThis indicates that the AI-assistant has successfully calculated the given mathematical expressions and provided the correct results.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 337, 'total_tokens': 443}, 'model_name': 'mistralai/mixtral-8x7b-instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-ab31b926-19b7-46e7-b9d1-fa6f9cd8ec03-0', usage_metadata={'input_tokens': 337, 'output_tokens': 106, 'total_tokens': 443})

Request: Few-shot prompting

In [19]:
llm_with_tools.invoke(
    "Whats 119 times 8 minus 20. Don't do any math yourself, only use tools for math. Respect order of operations"
).tool_calls


# [{'name': 'multiply',
#   'args': {'a': 119, 'b': 8},
#   'id': 'call_RofMKNQ2qbWAFaMsef4cpTS9'},
#  {'name': 'add',
#   'args': {'a': 952, 'b': -20},
#   'id': 'call_HjOfoF8ceMCHmO3cpwG6oB3X'}]

[]

In [20]:
# By adding a prompt with some examples we can correct this behavior 

from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, ToolMessage

examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
chain.invoke("Whats 119 times 8 minus 20")

# [{'name': 'multiply',
#   'args': {'a': 119, 'b': 8},
#   'id': 'call_tWwpzWqqc8dQtN13CyKZCVMe'}]

BadRequestError: Error code: 400 - {'error': {'message': 'Invalid message content.', 'type': 'invalid_request_error', 'param': 'messages.content', 'code': None}}

In [ ]:
chain.invoke("Whats 119 times 8 minus 20")


In [ ]:
# Giả sử bạn đã gọi công cụ và nhận được phản hồi
def process_tool_response(response):
    if 'tool_calls' in response:
        for tool_call in response['tool_calls']:
            print(f"Công cụ: {tool_call['name']}")
            print(f"Tham số: {tool_call['args']}")
    else:
        print("Phản hồi không chứa dữ liệu từ công cụ.")

# Xử lý phản hồi
process_tool_response(response)


Phản hồi không chứa dữ liệu từ công cụ.
